In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import annotations

import gc

import metatensor
import numpy as np
import torch

from halex.decomposition import EquivariantPCA
from halex.models import RidgeOnEnergiesAndLowdinMultipleMolecules  # ByMO
from halex.rotations import ClebschGordanReal
from halex.train_utils import (
    compute_features,
    coupled_fock_matrix_from_multiple_molecules,
    load_batched_dataset,
    load_molecule_scf_datasets,
)
from halex.utils import drop_target_samples

torch.set_default_dtype(torch.float64)

# Load dataset for both STO-3G and def2-TZVP basis

The dataset contains the frames, orbitals, Fock matrix, overlap matrix, orthogonalized Fock matrix and its block decomposition in `metatensor` format, eigenvalues and eigenvectors, and Löwdin charges in the minimal basis STO-3G and the fully converged def2-TZVP basis.

In [3]:
def load_molecule_datasets(base_folder:str, mol: str, cg: ClebschGordanReal, indices: np.ndarray):
    """
    Load the SCFData objects storing data for a single molecule,
    in both a small basis and a big basis
    """
    coords_path = f"{base_folder}/{mol}/coords_{mol}_1000.xyz"
    small_basis_path = f"{base_folder}/{mol}/b3lyp_STO-3G/"
    big_basis_path = f"{base_folder}/{mol}/b3lyp_def2tzvp/"

    sb_data, bb_data = load_molecule_scf_datasets(
        coords_path=coords_path,
        small_basis_path=small_basis_path,
        big_basis_path=big_basis_path,
        cg=cg,
        train_indices=indices,
    )

    return sb_data, bb_data

In [4]:
cg = ClebschGordanReal(4)

molecules = [
    "ethane",
    "ethene",
    # "butadiene",
    # "hexane",
    # "hexatriene",
    # "isoprene",
    # "styrene",
]

np.random.seed(12345)
indices = np.arange(1000)
np.random.shuffle(indices)
valid_indices = indices[-200:]
indices = indices[:500]
np.save("train_output/train_indices.npy", indices)
np.save("train_output/valid_indices.npy", valid_indices)

## Split the dataset into training and validation sets

In [5]:
datasets = {
    mol: load_molecule_datasets("CH-dataset", mol, cg=cg, indices=indices) for mol in molecules
}

valid_datasets = {
    mol: load_molecule_datasets("CH-dataset", mol, cg=cg, indices=valid_indices) for mol in molecules
}

# Compute features

To build features to learn the Hamiltonian we use the atom-centred equivariant representation $\ket{\overline{\rho_i ^{\otimes 2} ; \sigma ; \lambda \mu}}$ to learn the orbital interactions on the same centre, $H_{ii}$ terms and the two-centred equivariant representation $\ket{\overline{\rho_{ij} ^{\otimes 1} ; \sigma ; \lambda \mu}}$  to learn the the orbital interactions on two different centres, $H_{ij}$ terms. We have used the `librascal` library to compute these features and converted them to store in the `metatensor` format.

We then reduce the dimensionality of these features using PCA and retain up to 200 principal components for each symmetry block.







In [6]:
hypers = {
    "cutoff": 3.5,
    "max_radial": 6,
    "max_angular": 4,
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

feats = compute_features(datasets, hypers, cg=cg, lcut=2)
gc.collect()

epca = EquivariantPCA(n_components=200).fit(metatensor.join(feats, axis="samples"))

feats = [epca.transform(feats_) for feats_ in feats]
gc.collect()

epca.save("train_output/epca.npz")

valid_feats = compute_features(
    valid_datasets, hypers, cg=cg, lcut=2, epca=epca
)
gc.collect()

  0%|          | 0/2 [00:00<?, ?it/s]

/home/suman/miniconda3/envs/mlelec/lib/python3.10/site-packages/rascaline/systems/ase.py:57: UserWarning: periodic boundary conditions are disabled, but the cell matrix is not zero, we will set the cell to zero.
  warnings.warn(
100%|██████████| 2/2 [01:12<00:00, 36.49s/it]
fitting PCA on each tensormap key: 13it [00:06,  1.93it/s]
transforming each tensormap key: 13it [00:00, 67.34it/s]
transforming each tensormap key: 13it [00:00, 54.94it/s]
transforming each tensormap key: 13it [00:00, 146.19it/s]
transforming each tensormap key: 13it [00:00, 191.62it/s]
100%|██████████| 2/2 [00:26<00:00, 13.30s/it]


5375

# Batched dataset

In [7]:
nelec_dict = {"H": 1.0, "C": 6.0}

multimol_datasets = [
    load_batched_dataset(
        scf_datasets=data,
        feats=feat,
        nelec_dict=nelec_dict,
        batch_size=100,
        lowdin_charges_by_MO=False,
        # lowdin_mo_indices=indices,
    )
    for data, feat in zip(datasets.values(), feats)
]

valid_multimol_datasets = [
    load_batched_dataset(
        scf_datasets=data,
        feats=feat,
        nelec_dict=nelec_dict,
        batch_size=50,
        lowdin_charges_by_MO=False,
        # lowdin_mo_indices=indices,
    )
    for data, feat in zip(valid_datasets.values(), valid_feats)
]

#  Change to the Coupled basis

We convert the block decomposed Hamiltonian to a coupled basis and drop samples that are not present in the features.

In [8]:
targ_coupled = coupled_fock_matrix_from_multiple_molecules(datasets.values())
targ_coupled = drop_target_samples(
    metatensor.join(feats, axis="samples"), targ_coupled, verbose=True
)

all keys matched successfully


# Model Fitting

We build a symmetry adapted model of the minimal basis (STO-3G) Hamiltonian and train it against a fully converged def2-TZVP basis, using the eigenvalues and partial charges as indirect targets. The model is initialized from the weights obtained by the analytical solution of the ridge regression model (symmetry adapted) that uses as target the minimal basis Hamiltonian.

In [ ]:
model = RidgeOnEnergiesAndLowdinMultipleMolecules(
    coupled_tmap=targ_coupled,
    features=metatensor.join(feats, axis="samples"),
    alpha=1e-14,
    dump_dir="train_output",
    bias=False,
)

model.fit_ridge_analytical(
    features=metatensor.join(feats, axis="samples"),
    targets=targ_coupled,
)

In [10]:
model.fit(
    train_datasets=multimol_datasets,
    valid_datasets=valid_multimol_datasets,
    epochs=20_000,
    optim_kwargs=dict(lr=1),
    verbose=10,
    dump=50,
)

model.dump_state()

  2%|▊                                 | 480/20000 [19:10<13:00:04,  2.40s/it, train_total=7.36e+5, valid_total=5.43e+5]


KeyboardInterrupt: 